## SPAM DETECTION

# Spam Detection & Email Analysis

## Contesto
**ProfessionAI** ha richiesto lo sviluppo di una libreria per l'analisi delle email ricevute,
con focus sull'identificazione e l'analisi delle email di tipo SPAM.

## Obiettivi
Il progetto si articola in 4 task principali:

- **Classificazione SPAM** — addestrare un modello per identificare automaticamente le email spam
- **Topic Modeling** — individuare i topic principali tra le email SPAM
- **Distanza Semantica** — calcolare la distanza tra i topic per misurarne l'eterogeneità
- **NER (Named Entity Recognition)** — estrarre le organizzazioni dalle email NON SPAM

## Dataset
Dataset fornito da ProfessionAI contenente email etichettate come SPAM e non SPAM.

##  Tecnologie utilizzate
- **Python** (Pandas, Scikit-learn, NLTK/SpaCy)
- **Google Colab**

In [19]:
import pandas as pd

In [20]:
url = "https://raw.githubusercontent.com/EliaVenturini/professionai-projects/main/spam_dataset.csv"
dataset = pd.read_csv(url)

**Importazione e configurazione delle librerie**
Ho iniziato importando le librerie necessarie per l’elaborazione del testo, tra cui string, spacy, nltk, re, pandas, numpy, gensim e sklearn. Successivamente:

Ho caricato le stopwords in inglese utilizzando nltk.
Ho caricato il modello di word embeddings GloVe tramite gensim, per rappresentare le parole come vettori.
Ho inizializzato spaCy per poter gestire l’analisi linguistica e la lemmatizzazione del testo.
Definizione delle funzioni principali

- **Ho creato due funzioni chiave per la pulizia e la trasformazione del testo:**

  - **data_cleaner(sentence)**: Questa funzione ha convertito il testo in minuscolo, rimosso la punteggiatura (sostituendola con spazi), applicato la lemmatizzazione per ridurre le parole alla loro forma base, eliminato le stopwords e infine rimosso i numeri dal testo.

  - **avg_vector(sentence)**: Questa funzione calcolava il vettore medio della frase. Per ogni parola nella frase, se era presente nel modello GloVe, il suo vettore veniva sommato al vettore totale; altrimenti, veniva conteggiata come “non trovata”. Se nessuna parola era presente nel modello, la funzione restituiva un vettore di zeri; altrimenti, restituiva il vettore medio calcolato.
Caricamento e suddivisione del dataset

- **Ho caricato il dataset spam_dataset.csv.**
Dopo aver estratto la colonna label per ottenere le etichette SPAM e non-SPAM, ho suddiviso il dataset in un set di training (80%) e uno di test (20%) utilizzando train_test_split.

- **Pre-processing del testo nel dataset**
Per pulire i dati, ho applicato la funzione data_cleaner ai testi sia del set di training che di quello di test. Dopodiché, ho convertito il testo pulito in vettori numerici, utilizzando la funzione avg_vector per trasformare ogni frase nei set di training e test.

- **Addestramento del classificatore MLP**
Ho configurato e addestrato un Multi-Layer Perceptron (MLP) con:

 - Funzione di attivazione logistica
 - Un singolo strato nascosto con 100 neuroni
 - Metodo di ottimizzazione Adam
 - Tolleranza impostata a 0.005

Ho addestrato questo modello sui vettori di testo del set di training e sulle rispettive etichette di classificazione.

- **Estrazione delle organizzazioni nelle email non-SPAM**

Per identificare le organizzazioni citate nelle email non-SPAM:

Ho definito la funzione extract_organizations(text), che utilizzava spaCy per identificare le entità nominate nella frase e restituiva solo le entità con etichetta “ORG” (organizzazioni).
Ho quindi filtrato le email non-SPAM (etichettate come “ham”), applicato la pulizia del testo con data_cleaner, e utilizzato extract_organizations per estrarre e salvare le organizzazioni menzionate in queste email.

DATA CLEANING

In [21]:
import string  # Per gestire la punteggiatura
import spacy   # Per l'analisi del linguaggio naturale (lemmatizzazione)
import nltk    # Libreria per l'elaborazione del linguaggio naturale
from nltk.corpus import stopwords  # Importa le stopwords da nltk
import re      # Libreria per le espressioni regolari (regex)

nltk.download('stopwords') #dataset delle stopwords
english_stopwords = stopwords.words('english') #definisco le stopwords in inglese
nlp = spacy.load('en_core_web_sm') #carico il modello di Spacy per l'analisi del linguaggio (include lemmatizzazione)
punctuation = set(string.punctuation) #definisco la punteggiatura come un insieme

def data_cleaner(sentence):
    sentence = sentence.lower() #converti la frase in minuscolo per un'elaborazione coerente
    for c in string.punctuation:
      sentence=sentence.replace(c," ") #rimuovo la punteggiatura sostituendola con spazi
    document = nlp(sentence) #analizzo la frase con Spacy per applicare la lemmatizzazione
    sentence = " ".join(token.lemma_ for token in document) #lemmatizzo i token (restituisce la forma base di ogni parola)
    sentence = " ".join(word for word in sentence.split() if word not in english_stopwords) #rimuovo le stopwords (parole comuni come 'and', 'the')
    sentence = re.sub("\d", "", sentence) #uso espressioni regolari per rimuovere i numeri dalla frase
    return sentence #ritorna la stringa


<>:19: SyntaxWarning: invalid escape sequence '\d'
<>:19: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2867601852.py:19: SyntaxWarning: invalid escape sequence '\d'
  sentence = re.sub("\d", "", sentence) #uso espressioni regolari per rimuovere i numeri dalla frase
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
import pandas as pd
!pip install gensim
from gensim.models import Word2Vec
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

CALCOLO DEL VETTORE MEDIO DELLE FRASI

In [23]:
import numpy as np

#Funzione che calcola il vettore medio di una frase
def avg_vector(sentence):
    to_remove = 0 #Inizializzazione del contatore delle parole non trovate nei vettori GloVe
    vector = np.zeros(300)  #Inizializzione di un vettore di zeri di dimensione 300 (la dimensione dei vettori GloVe)

    # Itera su ogni parola nella frase (sentence è una lista di parole)
    for word in sentence:
        # Se la parola esiste nei vettori GloVe (controlla se è presente in 'glove_vectors')
        if word in glove_vectors.key_to_index.keys():
            # Somma il vettore della parola al vettore accumulato
            vector += glove_vectors.get_vector(word)
        else:
            # Se la parola non è presente nei vettori GloVe, aumenta il contatore di parole da rimuovere
            to_remove += 1

    # Se tutte le parole non sono presenti nei vettori GloVe, restituisce un vettore di zeri
    if len(sentence) == to_remove:
        return np.zeros(300)  # Restituisce un vettore di zeri

    #Restituisce il vettore medio dividendo il vettore accumulato per il numero di parole valide (non rimosse)
    return vector / (len(sentence) - to_remove)


CLASSIFICAZIONE DEL TESTO e ADDESTRAMENTO MODELLO

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

#carico dataset
url = "https://raw.githubusercontent.com/EliaVenturini/professionai-projects/main/spam_dataset.csv"
X = pd.read_csv(url)
y = X['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Applico data_cleaner
X_train['text'] = X_train['text'].apply(data_cleaner)
X_test['text'] = X_test['text'].apply(data_cleaner)

#Converte le righe di testo in vettori numerici usando avg_vector
X_train_vectors = np.array([avg_vector(sentence.split()) for sentence in X_train['text']])
X_test_vectors = np.array([avg_vector(sentence.split()) for sentence in X_test['text']])
#sentence.split() perchè la funzione avg_vector() accetta una lista iterando così
#su ogni parola della frase


clf = MLPClassifier(activation='logistic',
                    hidden_layer_sizes=(100,),
                    max_iter=100,
                    solver='adam',
                    tol=0.005,
                    verbose=True)

clf.fit(X_train_vectors, y_train)
#Il codice crea e allena una rete neurale di classificazione con un livello nascosto di 100 neuroni.
#Utilizza la funzione di attivazione logistica, l'ottimizzatore adam, e interrompe l'addestramento
#se non ci sono miglioramenti significativi nella perdita.

#Rete neurale di classificazione (MLPClassifier):
# (MLPClassifier) è un modello di classificazione che viene addestrato per distinguere due classi, come ad esempio spam e non-spam (detto anche ham).
#Una volta addestrato il modello con dati di esempio, questo modello può classificare nuove email in spam o non-spam.

Iteration 1, loss = 0.58260815
Iteration 2, loss = 0.52060436
Iteration 3, loss = 0.46206453
Iteration 4, loss = 0.40002664
Iteration 5, loss = 0.34142753
Iteration 6, loss = 0.29146758
Iteration 7, loss = 0.25309841
Iteration 8, loss = 0.22536051
Iteration 9, loss = 0.20410295
Iteration 10, loss = 0.18761112
Iteration 11, loss = 0.17435549
Iteration 12, loss = 0.16376060
Iteration 13, loss = 0.15393076
Iteration 14, loss = 0.14824484
Iteration 15, loss = 0.13994793
Iteration 16, loss = 0.13415294
Iteration 17, loss = 0.12856004
Iteration 18, loss = 0.12388906
Iteration 19, loss = 0.12021917
Iteration 20, loss = 0.11613106
Iteration 21, loss = 0.11279436
Iteration 22, loss = 0.11043640
Iteration 23, loss = 0.10731991
Iteration 24, loss = 0.10470676
Iteration 25, loss = 0.10230947
Iteration 26, loss = 0.10021856
Iteration 27, loss = 0.09815415
Iteration 28, loss = 0.09643895
Training loss did not improve more than tol=0.005000 for 10 consecutive epochs. Stopping.


MLPClassifier(activation='logistic', max_iter=100, tol=0.005, verbose=True)

In [27]:
# accuracy, precision, recall e F1-score
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test_vectors)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.98      0.98      0.98       742
        spam       0.94      0.94      0.94       293

    accuracy                           0.97      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.97      0.97      0.97      1035



APPLICARE IL MODELLO A NUOVE EMAIL

In [28]:
# Nuove email da classificare
new_emails = [
    "Hi, we have an amazing offer just for you! Click here to win big prizes!",
    "Please find the project report attached. Let me know if there are any questions.",
    "Your account has been selected for a special reward. Act now to claim it."
]

# 1. Pulizia delle nuove email
new_emails_cleaned = [data_cleaner(email) for email in new_emails]

# 2. Trasforma le nuove email in vettori
new_emails_vectors = np.array([avg_vector(sentence.split()) for sentence in new_emails_cleaned])

# 3. Classifica le nuove email utilizzando il modello allenato
predictions = clf.predict(new_emails_vectors)

# 4. Interpreta i risultati
for email, label in zip(new_emails, predictions):
    if label == 'spam':
        print(f"Email: {email}\nClassificazione: SPAM\n")
    else:
        print(f"Email: {email}\nClassificazione: NON-SPAM\n")
#zip(new_emails, predictions): Questa funzione "zippa" (accoppia) insieme gli elementi di new_emails
#(le email da classificare) e predictions (le predizioni del modello, cioè le etichette "spam" o "non-spam").

Email: Hi, we have an amazing offer just for you! Click here to win big prizes!
Classificazione: SPAM

Email: Please find the project report attached. Let me know if there are any questions.
Classificazione: NON-SPAM

Email: Your account has been selected for a special reward. Act now to claim it.
Classificazione: SPAM



ESTRAZIONE DELLE ORGANIZZAZIONE DALLE EMAIL NON SPAM

In [29]:
#Funzione per estrarre le organizzazioni

import spacy   # Per l'analisi del linguaggio naturale (lemmatizzazione)
#SpaCy: per l'elaborazione del linguaggio naturale, in particolare per estrarre le entità come le organizzazioni.
nlp = spacy.load('en_core_web_sm') #carico il modello di Spacy per l'analisi del linguaggio (include lemmatizzazione)


def extract_organizations(text):
    doc = nlp(text)
    organizations = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    return organizations

#Filtro le email non-SPAM
non_spam_emails = X[X['label'] == 'ham']['text']
non_spam_emails = non_spam_emails.apply(data_cleaner)
organizations = non_spam_emails.apply(extract_organizations)

#Estrazione delle organizzazioni nelle email non-SPAM:
#(extract_organizations) serve per analizzare il contenuto delle email classificate come non-spam e individuare le organizzazioni menzionate al loro interno.
#Questo processo utilizza la libreria spaCy (o simile) per l'estrazione di entità (NER - Named Entity Recognition).
#La funzione extract_organizations estrae i nomi di entità etichettate come ORG (organizzazioni) dai testi.


In [30]:
# TOPIC MODELING SULLE EMAIL SPAM
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Filtro solo le email SPAM
spam_emails = X[X['label'] == 'spam']['text']
spam_emails_cleaned = spam_emails.apply(data_cleaner)

# Vettorizzo il testo con CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5)
X_spam_matrix = vectorizer.fit_transform(spam_emails_cleaned)

# Addestro LDA con 5 topic
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X_spam_matrix)

# Mostro le top 10 parole per ogni topic
print(" Topic principali nelle email SPAM:\n")
feature_names = vectorizer.get_feature_names_out()
for i, topic in enumerate(lda.components_):
    top_words = [feature_names[j] for j in topic.argsort()[-10:]]
    print(f"Topic {i+1}: {', '.join(top_words)}")

 Topic principali nelle email SPAM:

Topic 1: online, time, www, new, want, good, com, get, http, subject
Topic 2: professional, microsoft, xp, office, subject, adobe, window, software, account, price
Topic 3: email, remove, please, subject, contact, www, message, nbsp, computron, com
Topic 4: inc, within, security, report, investment, may, information, stock, statement, company
Topic 5: color, tr, align, size, pill, width, height, http, td, font


In [31]:
# TOPIC MODELING SULLE EMAIL SPAM
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Filtro solo le email SPAM
spam_emails = X[X['label'] == 'spam']['text']
spam_emails_cleaned = spam_emails.apply(data_cleaner)

# Vettorizzo il testo con CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5)
X_spam_matrix = vectorizer.fit_transform(spam_emails_cleaned)

# Addestro LDA con 5 topic
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X_spam_matrix)

# Mostro le top 10 parole per ogni topic
print("Topic principali nelle email SPAM:\n")
feature_names = vectorizer.get_feature_names_out()
for i, topic in enumerate(lda.components_):
    top_words = [feature_names[j] for j in topic.argsort()[-10:]]
    print(f"Topic {i+1}: {', '.join(top_words)}")

🔍 Topic principali nelle email SPAM:

Topic 1: online, time, www, new, want, good, com, get, http, subject
Topic 2: professional, microsoft, xp, office, subject, adobe, window, software, account, price
Topic 3: email, remove, please, subject, contact, www, message, nbsp, computron, com
Topic 4: inc, within, security, report, investment, may, information, stock, statement, company
Topic 5: color, tr, align, size, pill, width, height, http, td, font


In [32]:
# DISTANZA SEMANTICA TRA I TOPIC
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calcolo il vettore medio per ogni topic usando GloVe
topic_vectors = []
for i, topic in enumerate(lda.components_):
    top_words = [feature_names[j] for j in topic.argsort()[-10:]]
    topic_vector = avg_vector(top_words)
    topic_vectors.append(topic_vector)

topic_vectors = np.array(topic_vectors)

# Calcolo la similarità coseno tra i topic
similarity_matrix = cosine_similarity(topic_vectors)

# Converto in distanza (1 - similarità)
distance_matrix = 1 - similarity_matrix

print(" Distanza semantica tra i topic (0=identici, 1=completamente diversi):\n")
for i in range(len(distance_matrix)):
    for j in range(i+1, len(distance_matrix)):
        print(f"Topic {i+1} ↔ Topic {j+1}: {distance_matrix[i][j]:.3f}")

 Distanza semantica tra i topic (0=identici, 1=completamente diversi):

Topic 1 ↔ Topic 2: 0.395
Topic 1 ↔ Topic 3: 0.312
Topic 1 ↔ Topic 4: 0.411
Topic 1 ↔ Topic 5: 0.628
Topic 2 ↔ Topic 3: 0.584
Topic 2 ↔ Topic 4: 0.377
Topic 2 ↔ Topic 5: 0.686
Topic 3 ↔ Topic 4: 0.600
Topic 3 ↔ Topic 5: 0.733
Topic 4 ↔ Topic 5: 0.835


## Conclusioni

L'analisi della distanza semantica tra i 5 topic identificati nelle email SPAM
evidenzia una **buona eterogeneità complessiva** dei contenuti.

Le coppie **Topic 3 ↔ Topic 5** (0.733) e **Topic 4 ↔ Topic 5** (0.835) mostrano
la maggiore distanza semantica, indicando che questi topic trattano argomenti
**molto diversi tra loro**.

Al contrario, **Topic 1 ↔ Topic 3** (0.312) e **Topic 1 ↔ Topic 2** (0.395)
risultano più simili, suggerendo una certa **sovrapposizione tematica**
probabilmente legata a pattern linguistici comuni nelle email SPAM
(es. offerte commerciali, premi, urgenza).

In generale, il modello LDA ha identificato topic **sufficientemente distinti**
da confermare l'eterogeneità dei contenuti SPAM analizzati.